In [131]:
! chmod 600 /content/kaggle.json

! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d venky73/spam-mails-dataset

  0% 0.00/1.86M [00:00<?, ?B/s]
100% 1.86M/1.86M [00:00<00:00, 132MB/s]


In [132]:
import zipfile
zip_file = zipfile.ZipFile('/content/spam-mails-dataset.zip')
zip_file.extractall('/tmp/')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

import string
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### IMPORT DATA

In [134]:
df = pd.read_csv('/tmp/spam_ham_dataset.csv')
df.drop(['label_num','Unnamed: 0'],axis=1, inplace=True)
df.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...


### Data Cleansing

In [135]:
# Cek apakah ada nilai Null
df.isnull().sum()

label    0
text     0
dtype: int64

In [139]:
# Membuat Aggregat
df['Count']=1
df[['label','Count']].groupby(['label'], as_index=False).count().shape[0]

2

In [140]:
# Cek Jumlah label / Persebaran label
df[['label','Count']].groupby(['label'],as_index=False).count().sort_values(['Count'], ascending=False).head(10)

,label,Count
0,ham,3672
1,spam,1499


In [194]:
import cleantext
# Fungsi Cleansing text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\':", "", text)
    text = re.sub(r"\'subject :", " ", text)
    #text = re.sub('\W', ' ', text)
    #text = re.sub('\s+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = cleantext.clean(text)
    # text = cleantext.clean_words(text)
    text = text.strip(' ')
    return text

In [195]:
list(df['text'][10:12].apply(clean_text))

['subject vocabl rnd word asceticismvcsc brand new stock attentionvocalscap inc stock symbol vcscvcsc top stock pick month april stock expect tobounc cent levelth stock hit time low bounc backstock go explod next day watch soarwatch stock go crazi next week break news vocalscap inc announc agreement resel mix networkservicescurr price expect project specul price next day expect project specul price next day vocalscap network inc build compani revolution thetelecommun industri afford phone system hardwar onlin softwar rate canada us vocalscap compani withglob reach receiv intern attent develop voiceov ip voip applic solut includ award win eyefontm asoftphon real time pc phone advanc implement pbxsystem compani call center itsp servic provid vocalscap creat softwar interact solut revolv aroundglob commun data voic converg compani use vocalscap forvoic internet protocol applic like ip pbx softswitch pc phoneand web phone provid real time human interact inform deliveryov internet vocalscap

In [197]:
df['textClean'] = df['text'].apply(clean_text)
df['textClean']= df['textClean'].str.replace('subject',"")
df.head()

,label,text,Count,textClean
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,1,enron methanol meter follow note gave monday ...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",1,hpl nom januari see attach file hplnol xl hpl...
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",1,neon retreatho ho ho around wonder time year ...
3,spam,"Subject: photoshop , windows , office . cheap ...",1,photoshop window offic cheap main trendingaba...
4,ham,Subject: re : indian springs\r\nthis deal is t...,1,indian springsthi deal book teco pvr revenu u...


In [203]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

X = df['textClean']



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [206]:
import re
cleaned_data=[]
for i in range(len(X)):
   text=re.sub('[^a-zA-Z]',' ',X.iloc[i])
   text=text.lower().split()
   text=[stemmer.stem(word) for word in text if (word not in stop_words)]
   text=' '.join(text)
   cleaned_data.append(text)

In [208]:
df['textClean'] = cleaned_data

In [209]:
# Cek Dataframe
df[['text','textClean','label']][6:12]


,text,textClean,label
6,Subject: spring savings certificate - take 30 ...,spring save certif take offsav use custom appr...,ham
7,Subject: looking for medication ? we ` re the ...,look medic best sourc difficult make materi co...,spam
8,Subject: noms / actual flow for 2 / 26\r\nwe a...,nom actual flow agr forward melissa jone texa ...,ham
9,"Subject: nominations for oct . 21 - 23 , 2000\...",nomin oct see attach file hplnl xl hplnl xl,ham
10,Subject: vocable % rnd - word asceticism\r\nvc...,vocabl rnd word asceticismvcsc brand new stock...,spam
11,Subject: report 01405 !\r\nwffur attion brom e...,report wffur attion brom est inst siupi pgst r...,spam


In [224]:
# Membuat Dataframe Baru / Data Clean
df_baru = pd.concat([df, df.label.apply(lambda x: ''.join(x)).str.get_dummies()], axis=1)
df_baru.drop(['text','label','Count'],axis=1, inplace=True)

In [225]:
df_baru

,textClean,ham,spam
0,enron methanol meter follow note gave monday p...,1,0
1,hpl nom januari see attach file hplnol xl hpln...,1,0
2,neon retreatho ho ho around wonder time year n...,1,0
3,photoshop window offic cheap main trendingaba ...,0,1
4,indian springsthi deal book teco pvr revenu un...,1,0
...,...,...,...
5166,put ftthe transport volum decrea oncontract th...,1,0
5167,follow nomshpl take extra mmcf weekend tri nex...,1,0
5168,calpin daili ga nominationjuli mention earlier...,1,0
5169,industri worksheet august activityattach works...,1,0


In [226]:
df_baru.columns

Index(['textClean', 'ham', 'spam'], dtype='object')

In [227]:
judul = df_baru['textClean'].values
label = df_baru[['ham', 'spam']].values

In [228]:
label

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]])

In [229]:
df_baru.head()

,textClean,ham,spam
0,enron methanol meter follow note gave monday p...,1,0
1,hpl nom januari see attach file hplnol xl hpln...,1,0
2,neon retreatho ho ho around wonder time year n...,1,0
3,photoshop window offic cheap main trendingaba ...,0,1
4,indian springsthi deal book teco pvr revenu un...,1,0


### Split Data

In [230]:
from sklearn.model_selection import train_test_split
judul_latih, judul_test, label_latih, label_test = train_test_split(judul, label, test_size=0.2,  random_state=42, shuffle=True)

In [231]:
# Tokenization
vocab_size = 10000
max_len = 100
trunc_type = "post"
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(judul_latih)

word_index = tokenizer.word_index

sequences_train = tokenizer.texts_to_sequences(judul_latih)
sequences_test = tokenizer.texts_to_sequences(judul_test)
pad_train = pad_sequences(sequences_train, maxlen=max_len, truncating=trunc_type)
pad_test = pad_sequences(sequences_test, maxlen=max_len, truncating=trunc_type)

print(pad_test.shape)

(1035, 100)


In [232]:
pad_train

array([[   0,    0,    0, ...,   36,   11, 2110],
       [   0,    0,    0, ...,    1,    5,    1],
       [   0,    0,    0, ...,   39, 2048,   39],
       ...,
       [   1,    1,    1, ...,    1,    1,    1],
       [   0,    0,    0, ...,    8,   50,  403],
       [   0,    0,    0, ...,  124,   11,  224]], dtype=int32)

In [234]:
# Model Embedding
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    tf.keras.layers.LSTM(128, dropout=0.5, recurrent_dropout=0.5, return_sequences=True),
    tf.keras.layers.LSTM(64, dropout=0.5, recurrent_dropout=0.5, return_sequences=False),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.005), metrics=['accuracy'], loss='categorical_crossentropy',)
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 100, 64)           640000    
                                                                 
 lstm_29 (LSTM)              (None, 100, 128)          98816     
                                                                 
 lstm_30 (LSTM)              (None, 64)                49408     
                                                                 
 dense_50 (Dense)            (None, 128)               8320      
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_51 (Dense)            (None, 2)                 258       
                                                                 
Total params: 796,802
Trainable params: 796,802
Non-t

In [235]:
#callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.75 and logs.get('val_accuracy')>0.75):
      self.model.stop_training = True
      print("\n akurasi dari training set and the validation set telah terpenuhi > 75%!")
callbacks = myCallback()

In [236]:
num_epochs = 25
history = model.fit(pad_train, label_latih, epochs=num_epochs, 
                    validation_data=(pad_test, label_test), verbose=2, callbacks=[callbacks])

Epoch 1/25

 akurasi dari training set and the validation set telah terpenuhi > 75%!
130/130 - 73s - loss: 0.2165 - accuracy: 0.9231 - val_loss: 0.0917 - val_accuracy: 0.9739 - 73s/epoch - 565ms/step
